In [1]:
!pip install --upgrade seaborn

     |████████████████████████████████| 285 kB 835 kB/s 
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.10.0
    Uninstalling seaborn-0.10.0:
      Successfully uninstalled seaborn-0.10.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import wandb
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
from os import listdir
from os.path import isfile,join
import yaml
os.chdir('/kaggle/working')
fold = 0
train_df = pd.read_csv(f'../input/siim-covid19-detection/train_image_level.csv')
from sklearn.model_selection import GroupKFold
df = train_df

gkf  = GroupKFold(n_splits = 5)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(df, groups = df.StudyInstanceUID.tolist())):
    df.loc[val_idx, 'fold'] = fold


In [3]:
train_df = df
train_df['image_path'] = f'/kaggle/input/siimcovid19-512-img-png-600-study-png/image/' + train_df.id + '.png'
classes = ['0. opacity']
train_files = []
val_files   = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
os.makedirs('/kaggle/working/siim-cov19/labels/train', exist_ok = True)
os.makedirs('/kaggle/working/siim-cov19/labels/val', exist_ok = True)
os.makedirs('/kaggle/working/siim-cov19/images/train', exist_ok = True)
os.makedirs('/kaggle/working/siim-cov19/images/val', exist_ok = True)
label_dir = '/kaggle/input/siim-covid-19-yolo-txt'
for file in tqdm(train_files):
    shutil.copy(file, '/kaggle/working/siim-cov19/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/siim-cov19/labels/train')
    
for file in tqdm(val_files):
    shutil.copy(file, '/kaggle/working/siim-cov19/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), '/kaggle/working/siim-cov19/labels/val')

In [4]:
classes = ['0. opacity']
from os import listdir
from os.path import isfile, join
import yaml
cwd='/kaggle/working/'
with open(join(cwd,'train.txt'),'w') as f:
    for path in glob('/kaggle/working/siim-cov19/images/train/*'):
        f.write(path+'\n')
        
with open(join(cwd,'val.txt'),'w') as f:
    for path in glob('/kaggle/working/siim-cov19/images/val/*'):
        f.write(path+'\n')
data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 1,
    names = classes
    )


with open(join( cwd , 'siim-cov19.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
    
f = open(join( cwd , 'siim-cov19.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- 0. opacity
nc: 1
train: /kaggle/working/train.txt
val: /kaggle/working/val.txt



In [5]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5','/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5')
import torch
from IPython.display import Image, clear_output
clear_output()
print('setup complete. using torch %s %s' %(torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

setup complete. using torch 1.6.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [6]:
!WANDB_MODE="dryrun" python train.py --img 512 --batch 24 --epochs 35 --data /kaggle/working/siim-cov19.yaml --hyp /kaggle/input/yolov5-1-yaml/hyp.scratch.yaml --weights yolov5x.pt --cache

2021-07-18 16:37:45.081991: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
100%|████████████████████████████████████████| 170M/170M [00:03<00:00, 54.3MB/s]

wandb: Offline run mode, not syncing to the cloud.
wandb: W&B is disabled in this directory.  Run `wandb on` to enable cloud syncing.
Scanning '/kaggle/working/siim-cov19/labels/train' for images and labels... 5068 found, 0 missing, 1611 empty, 0 corrupted: 100%|██████████| 5068/5068 [00:01<00:00, 2572.14it/s]
Scanning '/kaggle/working/siim-cov19/labels/train.cache' for images and labels... 5068 found, 0 missing, 1611 empty, 0 corrupted: 100%|██████████| 5068/5068 [00:00<00:00, 9841079.94it/s]
Caching images (4.0GB): 100%|██████████| 5068/5068 [00:19<00:00, 256.45it/s]
Scanning '/kaggle/working/siim-cov19/labels/val' for images and labels... 1266 found, 0 missing, 429 empty, 0 corrupted: 100%|██████████| 1266/1266 [00:00<00:00, 1286.40it/s]
Scanning '/kaggle/wor

In [7]:
# plt.figure(figsize=(20,20))
# plt.axis('off')
# plt.imshow(plt.imread('runs/train/exp3/labels_correlogram.jpg'))

In [8]:
!ls runs/train/exp

confusion_matrix.png				   test_batch0_pred.jpg
events.out.tfevents.1626626268.0758e17bfa64.105.0  test_batch1_labels.jpg
hyp.yaml					   test_batch1_pred.jpg
labels.jpg					   test_batch2_labels.jpg
labels_correlogram.jpg				   test_batch2_pred.jpg
opt.yaml					   train_batch0.jpg
precision_recall_curve.png			   train_batch1.jpg
results.png					   train_batch2.jpg
results.txt					   weights
test_batch0_labels.jpg


In [9]:
!python detect.py --weights 'runs/train/exp/weights/best.pt'\
--img 512\
--conf 0.1\
--iou 0.5\
--source /kaggle/working/siim-cov19/images/val\
--exist-ok

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.1, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='/kaggle/working/siim-cov19/images/val', update=False, view_img=False, weights=['runs/train/exp/weights/best.pt'])
Fusing layers... 
image 1/1266 /kaggle/working/siim-cov19/images/val/000a312787f2_image.png: 512x512 2 0. opacitys, Done. (0.036s)
image 2/1266 /kaggle/working/siim-cov19/images/val/0012ff7358bc_image.png: 512x512 2 0. opacitys, Done. (0.034s)
image 3/1266 /kaggle/working/siim-cov19/images/val/00326161e51e_image.png: 512x512 2 0. opacitys, Done. (0.033s)
image 4/1266 /kaggle/working/siim-cov19/images/val/0049814626c8_image.png: 512x512 2 0. opacitys, Done. (0.033s)
image 5/1266 /kaggle/working/siim-cov19/images/val/004cbd797cd1_image.png: 512x512 2 0. opacitys, Done. (0.033s)
image 6/1266 /kaggle/working/siim-cov19/images/val/00b8180bd3a8_image.png: 512x512 2 0. opacitys,